In [1]:
from aiida import load_profile  
load_profile()

Profile<uuid='1ad5c4ff2c1141ee9b4a511ac6859016' name='presto'>

In [2]:
from aiida.orm import load_code

qe_code = load_code("qe@localhost")

In [3]:
from aiida_workgraph import WorkGraph, task
from pathlib import Path
from ase.io import read, write, iread
import numpy as np
from aiida.orm import SinglefileData, Float, InstalledCode, List, Dict, KpointsData, StructureData, load_group
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_workgraph.manager import get_current_graph
from ase import units
import tempfile

@task.calcfunction(outputs=["scaled_file"])
def create_scales(
    structure_path,
    min_v: Float,
    max_v:Float,
    num_structs: int
): 

    atoms = read(structure_path.value)
    cell = atoms.get_cell()

    lattice_scalars = np.cbrt(np.linspace(min_v.value, max_v.value, num_structs.value))
    for i,s in enumerate(lattice_scalars):
        b = atoms.copy()
        b.set_cell(cell * s, scale_atoms=True)
        write("scaled.extxyz",b,append=i>0)

    return {
        "scaled_file": SinglefileData(Path("scaled.extxyz").resolve())
    }

@task.graph(outputs = ["structures"])
def qe(
    code: InstalledCode,
    kpoints_mesh: List,
    task_metadata: Dict,
    scaled_file: SinglefileData,
    ):

    wg = get_current_graph()

    kpoints = KpointsData()
    kpoints.set_kpoints_mesh(kpoints_mesh)

    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
    
    with scaled_file.as_path() as path:
        for i, structs in enumerate(iread(path, format="extxyz")):
            
            structure = StructureData(ase=structs)
            pseudos = pseudo_family.get_pseudos(structure=structure)

            ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                structure=structure,
                unit='Ry',
            )

            pw_params = {
                "CONTROL": {
                    "calculation": "scf",
                    'tprnfor': True,
                    'tstress': True,
                },
                "SYSTEM": {
                    "ecutwfc": ecutwfc,
                    "ecutrho": ecutrho,
                },
            }
            
            qe_task = wg.add_task(
                PwCalculation,
                code=code,
                parameters=pw_params,
                kpoints=kpoints,
                pseudos=pseudos,
                metadata=task_metadata.value,
                structure=structure,
            )

            wg.update_ctx({
                f"struct.{i}" :{
                    "trajectory":qe_task.outputs.output_trajectory,
                    "parameters": qe_task.outputs.output_parameters
                }
            })

    return {
        "structures": wg.ctx.struct,
    }

@task.calcfunction(outputs = ["test_file", "train_file", "valid_file"])
def create_train_files(structures):

    training_files = {}
    
    for i, structs in structures.items():
        tmpfile = "mlip.extxyz"

        for struct_out_params in structs.values():
            
            trajectory = struct_out_params["trajectory"]

            fileStructure = trajectory.get_structure(index=0)
            fileAtoms = fileStructure.get_ase()

            stress = trajectory.arrays["stress"][0]
            converted_stress = stress * units.GPa
            fileAtoms.info["qe_stress"] = converted_stress

            fileAtoms.info["units"] = {"energy": "eV","forces": "ev/Ang","stress": "ev/Ang^3"}
            fileAtoms.set_array("qe_forces", trajectory.arrays["forces"][0])

            parameters = struct_out_params["parameters"]
            fileParams = parameters.get_dict()
            fileAtoms.info["qe_energy"] = fileParams["energy"]
            write(tmpfile, fileAtoms, append=True)



In [4]:
scales_inputs = {
    "min_v": 0.95,
    "max_v": 1.05,
    "num_structs": 12
}

qe_inputs = {
    "task_metadata": Dict({
            "options": {
                "resources": {
                    "num_machines": 1,
                },
                "max_wallclock_seconds": 3600,
                "queue_name": "scarf",
                "qos": "scarf",
                "environment_variables": {},
                "withmpi": True,
                "prepend_text": """
                """,
                "append_text": "",
            },
    }),
    "kpoints_mesh": List([1, 1, 1]),
    "code": qe_code,
}

In [5]:
with WorkGraph("EOS_workflow") as wg:

    initial_structure = str(Path("../structures/NaCl-traj.xyz").resolve())
    
    scales_task = wg.add_task(
        create_scales,
        structure_path=initial_structure,
        **scales_inputs
    )

    qe_task = wg.add_task(
        qe,
        scaled_file=scales_task.outputs.scaled_file,
        **qe_inputs
    )

    train_task = wg.add_task(
        create_train_files,
        structures=qe_task.outputs.structures
    )
 

In [6]:
wg.run()

11/17/2025 12:30:15 PM <33412> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
11/17/2025 12:30:15 PM <33412> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
11/17/2025 12:30:15 PM <33412> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
11/17/2025 12:30:15 PM <33412> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1246|WorkGraphEngine|continue_workgraph]: tasks ready to run: create_scales
11/17/2025 12:30:16 PM <33412> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1246|WorkGraphEngine|update_task_state]: Task: create_scales, type: CALCFUNCTION, finished.
11/17/2025 12:30:16 PM <33412> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1246|WorkGraphEngine|continue_workgraph]: tasks ready to run: qe
11/17/2025 12:30:17 PM <33412> aiida

{}